In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
from scipy.stats import norm
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import datetime, timedelta
from PIL import Image
import selenium
import io

In [2]:
import numpy as np
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from numpy import cos, sin, arcsin, sqrt
from math import radians
import cartopy.io.img_tiles as cimgt
import folium

In [139]:
aja = 'data_spotter/aja.geojson'
ajb = 'data_spotter/ajb.geojson'
filename = 'data_spotter/pontos.csv'
rtd_filename = 'data_spotter/rtd.csv'

icon_url = "https://toppng.com/uploads/preview/lifebuoy-11530996451jlzqgbdrcd.png"

In [140]:
df = pd.read_csv(filename, delimiter=',')
rtd_df = pd.read_csv(rtd_filename, delimiter=',')

In [141]:
coordinates = []
for index, row in df.iterrows():
    coordinate = (row['LAT'], row['LON'])
    coordinates.append(coordinate)
df['coordinates'] = coordinates



In [142]:
coordinates = []
for index, row in rtd_df.iterrows():
    coordinate = (row['LAT'], row['LON'])
    coordinates.append(coordinate)
rtd_df['coordinates'] = coordinates


In [143]:
lat_s = int(np.trunc(rtd_df['LAT'].min()))
lat_n = int(np.ceil(rtd_df['LAT'].max()))
lon_w = int(np.trunc(rtd_df['LON'].min()))
lon_e = int(np.ceil(rtd_df['LON'].max()))
title_fig = 'Estações Costa Leste'
name_file = 'costa_leste'

In [167]:
m = folium.Map(location=[(lat_s+lat_n)/2, (lon_w+lon_e)/2], zoom_start=3)

popup1 = "Vessel Route"
folium.PolyLine(rtd_df['coordinates'], tooltip=popup1).add_to(m)


# icons = folium.features.CustomIcon(icon_url,
#                                   icon_size=(14, 14))


for index, row in df.iterrows():
    popup = f"Spotter #{row['ID']}"
    folium.Marker(row['coordinates'], tooltip=popup, icon=folium.Icon(color="red",icon="fa-life-ring", prefix='fa')).add_to(m)
    
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

style1 = {'fillColor': '#228B22', 'color': '#228B22'}
geojson1 = folium.GeoJson(ajb, name="geojson", style_function=lambda x:style1)
popup2 = folium.Popup('Brazilian Waters')
popup2.add_to(geojson1)
geojson1.add_to(m)

style2 = {'fillColor': '#228B22', 'color': '#FFFF00'}
geojson2 = folium.GeoJson(aja, name="geojson", style_function=lambda x:style2)
popup3 = folium.Popup('Argentine Waters')
popup3.add_to(geojson2)
geojson2.add_to(m)

folium.Circle([-37.2226,  -51.7676], popup='Wave formation area', fill_color='#000', radius=400000, weight=2, color="#000").add_to(m)
folium.Circle([-44.2767,  -58.6230], popup='Wave formation area', fill_color='#000', radius=400000, weight=2, color="#000").add_to(m)


m.add_child(folium.LatLngPopup())

m

m.save(outfile= "spotter_deployment.html")


In [145]:
arthur_df = pd.read_csv(arthur_filename, delimiter='\t')
rtd_arthur_df = pd.read_csv(rtd_arthur_filename, delimiter='\t')

coordinates = []
for index, row in arthur_df.iterrows():
    coordinate = (row['LAT'], row['LON'])
    coordinates.append(coordinate)
arthur_df['coordinates'] = coordinates
coordinates = []
for index, row in rtd_arthur_df.iterrows():
    coordinate = (row['LAT'], row['LON'])
    coordinates.append(coordinate)
rtd_arthur_df['coordinates'] = coordinates

m = folium.Map(location=[(lat_s+lat_n-2)/2, (lon_w+lon_e)/2], zoom_start=5.5)

folium.PolyLine(rtd_arthur_df['coordinates']).add_to(m)
for index, row in arthur_df.iterrows():
    popup = str(row['ID'])
    folium.Marker(row['coordinates'], tooltip=popup).add_to(m)
#     folium.CircleMarker(location=row['coordinates'],
#                         radius=5,
#                         fill=True,
#                         fill_color='red',
#                         color = 'grey',
#                         fill_opacity=0.7,
#                         weight=0).add_to(m)
    
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

popup1 = folium.LatLngPopup()

m.add_child(popup1)
m
m.save(outfile= "costa_leste.html")


NameError: name 'arthur_filename' is not defined

In [285]:
print(f"lat sul: {arthur_df['LAT'].min()}")
print(f"lat norte: {arthur_df['LAT'].max()}")
print(f"lon oeste: {arthur_df['LON'].min()}")
print(f"lon leste: {arthur_df['LON'].max()}")

lat sul: -21.672314672
lat norte: -13.0292565556
lon oeste: -40.1631372096
lon leste: -37.2548841154


In [54]:
import shapefile
from json import dumps

# read the shapefile
reader = shapefile.Reader("data_spotter/200Milhas/Linhas200milhas.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 
   
    # write the GeoJSON file
   
geojson = open("data_spotter/ajb.json", "w")
geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
geojson.close()

In [ ]:
f = open(ajb,)
x = json.load(f)

df1 = pd.DataFrame(np.array(x['features'][0]['geometry']['coordinates']))
df2 = pd.DataFrame(np.array(x['features'][1]['geometry']['coordinates']))

df1.columns = ['LON', 'LAT']
df2.columns = ['LON', 'LAT']

coordinates = []
for index, row in df1.iterrows():
    coordinate = (row['LAT']/100000, row['LON']/100000)
    coordinates.append(coordinate)
df1['coordinates'] = coordinates
coordinates = []
for index, row in df2.iterrows():
    coordinate = (row['LAT']/100000, row['LON']/100000)
    coordinates.append(coordinate)
df2['coordinates'] = coordinates